In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import multiprocessing as mp

import torch
from pytorch_model import Classifier

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print(mp.cpu_count(),' CPUs available')

Device: cuda
16  CPUs available


## Saving/Loading a pytorch model

In [3]:
# define model
model0 = Classifier(use_LSTM=True,N_metrics=5)
#model0 = model0.to(device)# put it on gpu

# save some test model weights
model_weights_path = 'training/model_weights_rand.pt'
torch.save(model0.state_dict(), model_weights_path)

In [4]:
# define a new model with the exact same architecture
model  = Classifier(use_LSTM=True,N_metrics=5)

# load it
model_weights_path = 'training/model_weights_rand.pt'
model.load_state_dict(torch.load(model_weights_path))
model.eval()

Classifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (text_read): LSTM(300, 128)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=133, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
  (softmax): Softmax(dim=1)
)

## Inputting text to the model

In [5]:
input_text_test = """My Amazing Adventure to Mars: The Search for Aliens and the Mysterious Face
Hey there, fellow explorers! Let me tell you about this incredible journey to Mars. Imagine a planet with red deserts and rocky landscapes, where scientists from NASA are sending cool rovers to find signs of life. And guess what? There's this mysterious mesa that people used to think looked like a giant face!
Back in 1976, NASA's Viking 1 spacecraft sent back a picture of a mesa on Mars that everyone thought was a face. It was like something out of a sci-fi movie! But later, when newer pictures came in, the face turned out to be just a funny-shaped hill. Still, the idea of finding a face on another planet got everyone super excited.
Now, why are we so obsessed with Mars? Well, NASA believes there might be tiny living things, like microbes, hiding under the surface or in old Martian lakes. How cool would that be? It's like playing hide and seek with aliens on a whole other planet!
The rovers NASA created, like Curiosity, are like our robotic buddies exploring Mars for us. These machines are super smart and have all sorts of tools to dig, sniff, and take awesome pictures. They're like our interplanetary detectives, searching for clues that could tell us if Mars ever had or still has any form of life.
Just think about it – if those rovers found even the tiniest hint of life on Mars, it would be mind-blowing! It would mean that we're not alone in this vast universe. It would be like discovering a whole new neighborhood of alien neighbors.
But getting to Mars and sending these cool rovers isn't easy. It takes a lot of brainpower from scientists and engineers who design and build these amazing space machines. They're like the superheroes of space exploration, using their smarts to unlock the secrets of the universe.
The idea of aliens has always been a big mystery that sparks our imagination. We wonder what they might look like and if they're friendly or not. Finding life on Mars would be like making friends with aliens, and that's just awesome to think about!
So, here's to the brave people at NASA and their rovers, exploring Mars and searching for signs of life. Who knows what they might discover next? Maybe they'll find something even cooler than a face – maybe they'll find our future Martian pals! It's like the best space adventure ever, and we're all a part of it, dreaming big and reaching for the stars.
"""

In [6]:
print(input_text_test)

My Amazing Adventure to Mars: The Search for Aliens and the Mysterious Face
Hey there, fellow explorers! Let me tell you about this incredible journey to Mars. Imagine a planet with red deserts and rocky landscapes, where scientists from NASA are sending cool rovers to find signs of life. And guess what? There's this mysterious mesa that people used to think looked like a giant face!
Back in 1976, NASA's Viking 1 spacecraft sent back a picture of a mesa on Mars that everyone thought was a face. It was like something out of a sci-fi movie! But later, when newer pictures came in, the face turned out to be just a funny-shaped hill. Still, the idea of finding a face on another planet got everyone super excited.
Now, why are we so obsessed with Mars? Well, NASA believes there might be tiny living things, like microbes, hiding under the surface or in old Martian lakes. How cool would that be? It's like playing hide and seek with aliens on a whole other planet!
The rovers NASA created, like C

### We need to do a little preprocessing 

(it might take a bit for some things to load)

In [7]:
%%time
from preprocessing import preprocess,  metrics, w2v
remap = {'a':'an', 'and':'also', 'of':'in', 'to':'at'}

def process_for_model(text):
    # extract metrics then do preprocessing
    met = metrics(text)
    
    # do some processing 
    proc = preprocess(text)
    processed_text_words = proc.split()# split by whitespace
    essay_words = [remap[word] if word in remap.keys() else word for word in processed_text_words]
    
    # do word2vec
    vecs = []
    missing_words =[]
    for word in essay_words:
        try:
            vec = w2v.get_vector(word)
            vecs.append(vec)
        except KeyError:
            # this means that the word isn't in the w2v
            missing_words.append(word)
    vecs = np.array(vecs)
    

    #print all the words that are missing
    unique_missing =  " ".join(list(np.unique(np.array(missing_words))))
    print('missing these words in model: ',unique_missing)
    
    torch_tensor = torch.from_numpy(vecs).reshape(-1,300)
    torch_met = torch.from_numpy(met).float()
    return torch_tensor, torch_met

[nltk_data] Downloading package punkt to /home/rachel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rachel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/rachel/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [8]:
%%time
tensor,metrics = process_for_model(input_text_test)

missing these words in model:  
CPU times: user 69.8 ms, sys: 20 ms, total: 89.8 ms
Wall time: 88.6 ms


In [9]:
tensor, tensor.shape

(tensor([[ 0.1445,  0.0471,  0.1006,  ...,  0.0023,  0.0981, -0.0669],
         [ 0.0737,  0.0041, -0.1357,  ..., -0.1416, -0.1504, -0.1377],
         [ 0.2002, -0.0197, -0.1357,  ...,  0.0928,  0.2012,  0.1318],
         ...,
         [-0.0118, -0.0474,  0.0447,  ...,  0.0713, -0.0349,  0.0242],
         [ 0.0801,  0.1050,  0.0498,  ...,  0.0037,  0.0476, -0.0688],
         [-0.0184,  0.1709,  0.1108,  ..., -0.1348, -0.0162,  0.0167]]),
 torch.Size([449, 300]))

In [10]:
metrics

tensor([6.3545e+01, 2.4300e-01, 2.1000e-02, 7.3600e-01, 9.9880e-01])

### Get a prediction

In [15]:
n_inputs = 1# number of text corpuses being inputted
predictions = model(tensor.reshape(-1,n_inputs,300), metrics.reshape(n_inputs,-1))

print(predictions)

tensor([[0.5318, 0.4682]], grad_fn=<SoftmaxBackward0>)


In [22]:
pred_GPT = predictions[0][0].item()
pred_human = predictions[0][1].item()

print('The probability the text was Chat GPT created is ', pred_GPT)
print('The probability the text was human created is ', pred_human)

The probability the text was Chat GPT created is  0.5318042635917664
The probability the text was human created is  0.4681957960128784
